In [1]:
# Constants
from config.execution_parameters import *

# Project libraries
from src.simulator_package.simulator_functions import *
import src.hmm_package.generate_hmm
from src.hmm_package.generate_hmm import *
from src.plot_and_print_info.plots_and_print_info import *

import random
import importlib

In [75]:
importlib.reload(src.hmm_package.generate_hmm)
from src.hmm_package.generate_hmm import *

In [3]:
advertising_campaign = SimulationClass(ANALYSIS_MODE)
advertising_campaign.simulate()

adstock = compute_adstock(observation=advertising_campaign.results["user_expositions"])
output = np.append( np.zeros([adstock.shape[0], 1]), advertising_campaign.results["user_outcome"], axis=1 )

2022-03-23 00:11:08.956039: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
adstock.shape

TensorShape([12188, 3, 31])

In [5]:
empty_users = 6000
adstock_with_empty = tf.concat([adstock, tf.zeros([empty_users, adstock.shape[1], adstock.shape[2]], dtype=float)], axis = 0)
output_with_empty = np.append(output, np.zeros([empty_users, adstock.shape[2]]), axis = 0)

(12188, 31)

In [8]:
empty_users = 5000
adstock_only_empty = tf.zeros([empty_users, adstock.shape[1], adstock.shape[2]], dtype=float)
output_only_empty = np.zeros([empty_users, adstock.shape[2]])

# Lets add a 3% of conversion due to external factors
for temp in range(round(0.03*empty_users)):
    output_only_empty[temp, random.randint(1,adstock.shape[2]):] = 1


In [48]:

model = build_hmm_to_fit( states_observable=STATES_ARE_OBSERVABLE )

compiler = CompilerInfo(LR_EXPONENTIAL_DECAY)
model.compile(
    loss = compiler.loss,
    optimizer = compiler.optimizer,
    run_eagerly = True
)

#fit_model(model, adstock, output)
fit_model(model, adstock_with_empty, output_with_empty)

Epoch 1/10000
364/364 [==============================] - 254s 698ms/step - loss: 1.3159
Epoch 2/10000
364/364 [==============================] - 251s 691ms/step - loss: 1.2895
Epoch 3/10000
364/364 [==============================] - 252s 693ms/step - loss: 1.2599
Epoch 4/10000
364/364 [==============================] - 251s 689ms/step - loss: 1.2308
Epoch 5/10000
364/364 [==============================] - 251s 689ms/step - loss: 1.2129
Epoch 6/10000
364/364 [==============================] - 252s 691ms/step - loss: 1.1869
Epoch 7/10000
364/364 [==============================] - 251s 689ms/step - loss: 1.1431
Epoch 8/10000
364/364 [==============================] - 251s 689ms/step - loss: 1.0793
Epoch 9/10000
364/364 [==============================] - 251s 689ms/step - loss: 1.0026
Epoch 10/10000
364/364 [==============================] - 250s 688ms/step - loss: 0.9232
Epoch 11/10000
364/364 [==============================] - 251s 688ms/step - loss: 0.8479
Epoch 12/10000
364/364 [======

KeyboardInterrupt: 

In [53]:
fit_model(model, adstock_with_empty, output_with_empty)

Epoch 1/10000
364/364 [==============================] - 249s 685ms/step - loss: 0.2966
Epoch 2/10000
364/364 [==============================] - 249s 685ms/step - loss: 0.2966
Epoch 3/10000
364/364 [==============================] - 250s 688ms/step - loss: 0.2966
Epoch 4/10000
364/364 [==============================] - 241s 661ms/step - loss: 0.2965
Epoch 5/10000
364/364 [==============================] - 241s 661ms/step - loss: 0.2965
Epoch 6/10000
175/364 [=============>................] - ETA: 2:06 - loss: 0.3002

KeyboardInterrupt: 

In [31]:
t = make_transition_matrix(model.weights[0], model.weights[1], adstock_with_empty[-2:-1,:,:])
hmm = tfd.HiddenMarkovModel(
            initial_distribution=generate_hmm_distributions(transition_matrix=t, states_observable=False)['initial_distribution'],
            transition_distribution=tfd.Categorical(probs=t),
            observation_distribution=generate_hmm_distributions(transition_matrix=t, states_observable=False)['observation_distribution'],
            time_varying_transition_distribution=True,
            num_steps=execution_duration + 1)

In [77]:
make_non_exposed_user_transtion_matrix(model.weights[0],1000)

<tf.Tensor: shape=(30, 3, 3), dtype=float32, numpy=
array([[[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
        [1.0000000e-06, 1.0000000e-06, 9.9999797e-01]],

       [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
        [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
   

In [54]:
make_transition_matrix(model.weights[0], model.weights[1], adstock[0:1,:,:])

<tf.Tensor: shape=(1, 30, 3, 3), dtype=float32, numpy=
array([[[[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
         [1.0000000e-10, 1.0000000e-10, 1.0000000e+00]],

        [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
         [1.0000000e-10, 1.0000000e-10, 1.0000000e+00]],

        [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
         [1.0000000e-10, 1.0000000e-10, 1.0000000e+00]],

        [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
         [1.0000000e-10, 1.0000000e-10, 1.0000000e+00]],

        [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01, 5.0044274e-03],
         [1.0000000e-10, 1.0000000e-10, 1.0000000e+00]],

        [[9.9954051e-01, 3.6151949e-04, 9.7974400e-05],
         [4.9749774e-01, 4.9749786e-01,

In [52]:
model.weights

[<tf.Variable 'transition_prob_layer_5/mu:0' shape=(4,) dtype=float32, numpy=array([-7.3066144, -9.215984 , -0.       , -4.5944905], dtype=float32)>,
 <tf.Variable 'transition_prob_layer_5/beta:0' shape=(12,) dtype=float32, numpy=
 array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       dtype=float32)>]

In [80]:
a = make_transition_matrix(model.weights[0], model.weights[1], adstock_with_empty[-2:-1,:,:])
np.linalg.matrix_power( a[0,0,:].numpy(), 30 )

array([[9.9624962e-01, 7.1688538e-04, 3.0336005e-03],
       [9.8652732e-01, 7.0989016e-04, 1.2762990e-02],
       [5.7610592e-09, 2.0286492e-10, 1.0000000e+00]], dtype=float32)

In [ ]:
output_with_empty

In [82]:
np.linalg.matrix_power( a[0,0,:].numpy(), 30 )[0,2]

0.0030336005